In [1]:
import numpy as np
import pandas as pd

导入数据

In [2]:
train_data = pd.read_csv("../data/train.tsv",sep='\t')
test_data = pd.read_csv("../data/test.tsv",sep='\t')

In [3]:
train_data.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [5]:
train_data.shape

(156060, 4)

In [6]:
test_data.head()

,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine ...
1,156062,8545,An intermittently pleasing but mostly routine ...
2,156063,8545,An
3,156064,8545,intermittently pleasing but mostly routine effort
4,156065,8545,intermittently pleasing but mostly routine


In [7]:
test_data.shape

(66292, 3)

构建语料库

In [3]:
train_sentences = train_data['Phrase']
test_sentences = test_data['Phrase']
sentences = pd.concat([train_sentences,test_sentences])

In [12]:
sentences.head()
sentences.shape

(222352,)

In [4]:
label = train_data['Sentiment']

In [14]:
label

0         1
1         2
2         2
3         2
4         2
         ..
156055    2
156056    1
156057    3
156058    2
156059    2
Name: Sentiment, Length: 156060, dtype: int64

In [5]:
stop_words = open('../../archive/stop_words/english.txt').read().splitlines()

stop_words 导入为1个列表

In [6]:
stop_words

["'ll",
 "'tis",
 "'twas",
 "'ve",
 '10',
 '39',
 'a',
 "a's",
 'able',
 'ableabout',
 'about',
 'above',
 'abroad',
 'abst',
 'accordance',
 'according',
 'accordingly',
 'across',
 'act',
 'actually',
 'ad',
 'added',
 'adj',
 'adopted',
 'ae',
 'af',
 'affected',
 'affecting',
 'affects',
 'after',
 'afterwards',
 'ag',
 'again',
 'against',
 'ago',
 'ah',
 'ahead',
 'ai',
 "ain't",
 'aint',
 'al',
 'all',
 'allow',
 'allows',
 'almost',
 'alone',
 'along',
 'alongside',
 'already',
 'also',
 'although',
 'always',
 'am',
 'amid',
 'amidst',
 'among',
 'amongst',
 'amoungst',
 'amount',
 'an',
 'and',
 'announce',
 'another',
 'any',
 'anybody',
 'anyhow',
 'anymore',
 'anyone',
 'anything',
 'anyway',
 'anyways',
 'anywhere',
 'ao',
 'apart',
 'apparently',
 'appear',
 'appreciate',
 'appropriate',
 'approximately',
 'aq',
 'ar',
 'are',
 'area',
 'areas',
 'aren',
 "aren't",
 'arent',
 'arise',
 'around',
 'arpa',
 'as',
 'aside',
 'ask',
 'asked',
 'asking',
 'asks',
 'associated

词袋模型：失去了词的顺序

TF-IDF模型
TF term frequency
IDF inverse document frequency

Word2Vec

#1 词袋模型

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
co = CountVectorizer(
    analyzer='word',
    ngram_range=(1,4),
    stop_words=stop_words,
    max_features=150000
)

#ngram 几个词语连在一起一起进去，最多十五万词

In [8]:
co.fit(sentences)

/Users/erlich_jaso/opt/anaconda3/envs/pytorch/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'daren', 'hadn', 'herse', 'himse', 'itse', 'mayn', 'mightn', 'mon', 'mustn', 'myse', 'needn', 'oughtn', 'shan'] not in stop_words.
  warnings.warn(


CountVectorizer(max_features=150000, ngram_range=(1, 4),
                stop_words=["'ll", "'tis", "'twas", "'ve", '10', '39', 'a',
                            "a's", 'able', 'ableabout', 'about', 'above',
                            'abroad', 'abst', 'accordance', 'according',
                            'accordingly', 'across', 'act', 'actually', 'ad',
                            'added', 'adj', 'adopted', 'ae', 'af', 'affected',
                            'affecting', 'affects', 'after', ...])

In [9]:
from sklearn.model_selection import train_test_split
train_X,test_X,train_y,test_y = train_test_split(train_sentences,label,random_state=1234)

In [21]:
train_X[1]

'A series of escapades demonstrating the adage that what is good for the goose'

In [22]:
train_y[1]

2

将每一个词向量化

In [10]:
train_X = co.transform(train_X)
test_X = co.transform(test_X)

In [24]:
train_X[1]

<1x150000 sparse matrix of type '<class 'numpy.int64'>'
	with 6 stored elements in Compressed Sparse Row format>

忽略所有警告

In [11]:
import warnings
warnings.filterwarnings("ignore")

逻辑回归模型 训练

In [12]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(train_X,train_y)

LogisticRegression()

检验正确率

In [13]:
LR.score(test_X,test_y)

0.6453671664744329

朴素贝叶斯分类器

In [28]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(train_X,train_y) 

MultinomialNB()

In [29]:
classifier.score(test_X,test_y)

0.608714596949891

训练速度较快 但是准确率低一点

#2 TFIDF模型

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(
    analyzer='word',
    ngram_range=(1,4),
    max_features=150000
)
tf.fit(sentences)

TfidfVectorizer(max_features=150000, ngram_range=(1, 4))

In [19]:
train_X,test_X,train_y,test_y = train_test_split(train_sentences,label,random_state=1234)
train_X = tf.transform(train_X)
test_X = tf.transform(test_X)

LR.fit(train_X,train_y)
LR.score(test_X,test_y)

0.6407791874919903

In [35]:
classifier.fit(train_X,train_y) 
classifier.score(test_X,test_y)

0.6045367166474432

调整超参数

提交

In [15]:
X = co.transform(train_sentences)
LR.fit(X,label)

LogisticRegression()

In [16]:
LR.score(test_X,test_y)

0.7503267973856209

In [21]:
test_data_transform = co.transform(test_data['Phrase'])


In [23]:
prediction = LR.predict(test_data_transform)

In [26]:
prediction.shape

(66292,)

In [27]:
test_data.loc[:,'Sentiment'] = prediction

In [28]:
test_data.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,156061,8545,An intermittently pleasing but mostly routine ...,3
1,156062,8545,An intermittently pleasing but mostly routine ...,3
2,156063,8545,An,2
3,156064,8545,intermittently pleasing but mostly routine effort,3
4,156065,8545,intermittently pleasing but mostly routine,3


In [36]:
output  =test_data.loc[:,['PhraseId','Sentiment']]

In [37]:
output.head()

,PhraseId,Sentiment
0,156061,3
1,156062,3
2,156063,2
3,156064,3
4,156065,3


In [40]:
output.to_csv('../output/output.csv',index=None)